In [1]:
import os

In [2]:
%pwd

'd:\\Silent_Night\\mlops\\AML_Classification'

In [3]:
from AML_Classifier import logger
from AML_Classifier.pipeline.stage_01_data_ingestion import DataIngestionTrainingPipeline

In [5]:
STAGE_NAME="Data Ingestion stage"

try:
    logger.info(f">>>>>>> Stage {STAGE_NAME} started <<<<<<<")
    obj = DataIngestionTrainingPipeline()
    obj.main()
    logger.info(f">>>>>>> Stage {STAGE_NAME} Completed <<<<<<< \n x=========================================x")
    
except Exception as e:
    logger.exception(e)
    raise e 

[2024-06-27 15:28:30,674: INFO: 3440599255: >>>>>>> Stage Data Ingestion stage started <<<<<<<]
[2024-06-27 15:28:31,053: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-27 15:28:31,070: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-27 15:28:31,076: INFO: common: created directory at: artifacts]
[2024-06-27 15:28:31,076: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-27 15:28:31,719: INFO: credentials: Found credentials in shared credentials file: ~/.aws/credentials]
[2024-06-27 15:28:36,317: INFO: data_ingestion: Downloaded data from amaterasu AWS bucket into the path artifacts/data_ingestion/data.csv]
[2024-06-27 15:28:36,336: INFO: 3440599255: >>>>>>> Stage Data Ingestion stage Completed <<<<<<< 
 x=========================================x]


In [3]:
import pandas as pd
import numpy as np
import pickle

class PredictionPipeline:
    def __init__(self,pred_data):
        self.pred_data=pd.DataFrame([pred_data])

    def predict(self):
        #Preprocess the data
        #load the standard scaler
        with open("artifacts/model_data/scaler_obj.pkl","rb") as file:
            sc = pickle.load(file)

        pred_data_sc=sc.transform(self.pred_data[['from_bank', 'to_bank', 'amount_received', 'amount_paid']])
        pred_data_sc=pd.DataFrame(pred_data_sc, columns=['from_bank', 'to_bank', 'amount_received', 'amount_paid'])

        #load the label encoders
        pred_data_le=pd.DataFrame()
        for i in ['receiving_currency', 'Payment_Currency', 'Payment Format']:
            with open(f"artifacts/model_data/le_{i}.pkl","rb") as file:
                le=pickle.load(file)
            pred_data_le[i]=le.transform(np.array(self.pred_data[[i]]).ravel())

        #Concate the two scaled data
        pred_data_scaled=pd.concat([pred_data_le,pred_data_sc],axis=1)

        #Load the model
        with open("artifacts/prepare_base_model/base_model.pkl","rb") as file:
            model = pickle.load(file)

        #predict the output
        result=model.predict(pred_data_scaled)

        #Return the result
        if result[0]==1:
            prediction="Laundering Transaction"
            return [{"Prediction": prediction}]
        else:
            prediction = "Normal Transaction"
            return [{"Prediction": prediction}]

In [17]:
# Dictionary to be converted
test_data = {
    'receiving_currency' : "US Dollar",
    'Payment_Currency' : "US Dollar",
    'Payment Format' : "Cash",
    'from_bank': 70,
    'to_bank': 22661,
    'amount_received': 70831.64,
    'amount_paid' : 70831.64
}

# Convert dictionary to DataFrame
#df_test = pd.DataFrame([test_data])

# Print the DataFrame
#print(df_test)


In [26]:
cl=PredictionPipeline(test_data)

In [15]:
cl.__init__(test_data)

In [27]:
x=cl.predict()
print(x)

[{'Prediction': 'Laundering Transaction'}]


In [21]:
class ClientApp:
    def __init__(self,x_data):
        #self.pred_data = pred_data
        self.classifier = PredictionPipeline(pred_data=x_data)

    def predict(self):
        return self.classifier.predict()

In [24]:
clapp=ClientApp(test_data)

In [25]:
clapp.predict()

[{'Prediction': 'Laundering Transaction'}]